### Apprentissage 

In [1]:
import keras
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.utils import np_utils, generic_utils
from keras.optimizers import SGD, RMSprop, Adadelta, Adagrad, Adam
from keras.layers.convolutional import Conv2D, MaxPooling2D,Conv1D, MaxPooling1D
from keras.models import model_from_config
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.models import load_model
from keras.layers.convolutional import AveragePooling2D
from keras.callbacks import EarlyStopping
import RNA
from sklearn.metrics import roc_curve, auc
import numpy as np

import random
import gzip
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from scipy import sparse
import pdb
from math import  sqrt
from sklearn.metrics import roc_curve, auc
import theano
import subprocess as sp
import scipy.stats as stats
import argparse
import pandas as pd
import csv
from random import shuffle

/home/louis_jeay/downloads/3-5-2-test/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [15]:
def clean(seq):
    seq = seq.split('\n')
    seq2 = ''
    for j in seq:
        seq2 = seq2 + j
    seq2 = seq2[1:len(seq2)-1]
    seq2 = seq2.split(' ')
    #print(seq2)
    seq3=[]
    for j in seq2:
        #print(j)
        if j=='':
            a=0
        else:
            seq3.append(float(j))
    return seq3

def load_data(nb_train = 134000,nb_val = 6000,nb_test = 25000, nb_benchmark=5000):
    # load your data using this function
    verif = []

    verif = pd.read_csv("genomes/human_divers_negatifs.csv", sep = "\t",header=None)
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")            
    neg1 = verif
    verif=[]

    verif = pd.read_csv("genomes/human_mRNA-mRNA_negatifs.csv", sep = "\t",header=None)
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
    neg2 = verif
    verif=[]
    verif = pd.read_csv("genomes/mouse_divers_neg.csv", sep = "\t",header=None)
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
    neg3 = verif
    verif=[]

    verif = pd.read_csv("genomes/human_divers.csv", sep = "\t",header=None)  
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            
    pos1 = verif
    l = len(pos1)
    verif=[]

    verif = pd.read_csv("genomes/human_mRNA-mRNA.csv", sep = "\t",header=None)  
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            
    pos2 = verif
    l=l+len(pos2)
    verif=[]

    verif = pd.read_csv("genomes/mouse_divers_pos.csv", sep = "\t",header=None)  
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            
    pos3 = verif
    l=l+len(pos3)
    verif=[]

    bdd = np.concatenate((pos1,pos2,pos3,neg1,neg2,neg3))

    pos1 = []
    neg1 = []
    pos2 = []
    neg2 = []
    pos3 = []
    neg3 = []

    labels = np.zeros((len(bdd),1))

    bdd = np.concatenate((bdd,labels),axis=1)
    for i in range(l):
        bdd[i,6]=1
    labels=[]
    
    # shuffle total

    indices = np.arange(len(bdd))
    shuffle(indices)
    bdd = bdd[indices]
    indices=[]
    print(len(bdd))
    # 
    l = len(bdd) - nb_benchmark
    matrice = np.zeros((l,36,36,1)) 
    matrice1 = np.zeros((l,36,36,1))
    matrice2 = np.zeros((l,36,4))
    matrice3 = np.zeros((l,36,4))
    matrice4 = np.zeros((l,36,4))
    matrice5 = np.zeros((l,36,4))

    for i in range(l):
        seq1 = bdd[i,0][85:121]
        seq2 = bdd[i,1][85:121]
        prob1 = np.array(clean(bdd[i,2]))
        prob2 = np.array(clean(bdd[i,3]))
        for j in range(len(seq1)):
            for k in range(len(seq2)):
                if (seq1[j]=='a' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='a'):
                    matrice[i,j,k,0] = 1
                elif (seq1[j]=='g' and seq2[k]=='c') or (seq1[j]=='c' and seq2[k]=='g'):
                    matrice[i,j,k,0] = 1
                elif (seq1[j]=='g' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='g'):
                    matrice[i,j,k,0] = 1
                matrice1[i,j,k,0] = prob1[j]+prob2[k]
        for j in range(len(seq1)):
            if seq1[j]=='a':
                matrice2[i,j,0] = 1
            elif seq1[j]=='u':
                matrice2[i,j,1] = 1
            elif seq1[j]=='g':
                matrice2[i,j,2] = 1
            elif seq1[j]=='c':
                matrice2[i,j,3] = 1
        for j in range(len(seq2)):
            if seq2[j]=='a':
                matrice3[i,j,0] = 1
            elif seq2[j]=='u':
                matrice3[i,j,1] = 1
            elif seq2[j]=='g':
                matrice3[i,j,2] = 1
            elif seq2[j]=='c':
                matrice3[i,j,3] = 1

    for i in range(36):
        matrice4[:,36-i-1,:] = matrice2[:,i,:]
    for i in range(36):
        matrice5[:,36-i-1,:] = matrice3[:,i,:]
    
    training = []
    training.append(matrice[:nb_train])
    training.append(matrice1[:nb_train])
    training.append(matrice2[:nb_train])
    training.append(matrice4[:nb_train])
    training.append(matrice3[:nb_train])
    training.append(matrice5[:nb_train])    
    
    validation = []
    validation.append(matrice[nb_train:nb_train+nb_val])
    validation.append(matrice1[nb_train:nb_train+nb_val])
    validation.append(matrice2[nb_train:nb_train+nb_val])
    validation.append(matrice4[nb_train:nb_train+nb_val])
    validation.append(matrice3[nb_train:nb_train+nb_val])
    validation.append(matrice5[nb_train:nb_train+nb_val])

    testing = []
    testing.append(matrice[-nb_test:])
    testing.append(matrice1[-nb_test:])
    testing.append(matrice2[-nb_test:])
    testing.append(matrice4[-nb_test:])
    testing.append(matrice3[-nb_test:])
    testing.append(matrice5[-nb_test:])
    
    data_benchmark = bdd[-nb_benchmark:]
    
    labels = bdd[:l,6]
    bdd = []
    y = labels[:nb_train]
    y = keras.utils.np_utils.to_categorical(y,2)
    val_y = labels[nb_train:nb_train+nb_val]
    val_y = keras.utils.np_utils.to_categorical(val_y,2)
    true_y = labels[-nb_test:]
    argtest = []
    np.sum(y[:,1])
    return training, y, validation, val_y, testing, true_y, data_benchmark

In [16]:
def calculate_performance(test_num, pred_y,  labels):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for index in range(test_num):
        if labels[index] ==1:
            if labels[index] == pred_y[index]:
                tp = tp +1
            else:
                fn = fn + 1
        else:
            if labels[index] == pred_y[index]:
                tn = tn +1
            else:
                fp = fp + 1               
            
    acc = float(tp + tn)/test_num
    precision = float(tp)/(tp+ fp)
    sensitivity = float(tp)/ (tp+fn)
    specificity = float(tn)/(tn + fp)
    MCC = float(tp*tn-fp*fn)/(np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)))
    return acc, precision, sensitivity, specificity, MCC 

In [17]:
def perfs_test(model,test,true):
    print('predicting')
            
    predictions = model.predict(test)[:,1]
    print(predictions)
    for i,nulll in enumerate(predictions):
        predictions[i] = round(predictions[i])
    print(predictions,true)
    perfs = calculate_performance(len(predictions), predictions, true)
    
    print("acc : ", perfs[0])
    print("precision : ", perfs[1])
    print("sensitivity : ", perfs[2])
    print("specificity : ", perfs[3])
    print("MCC : ", perfs[4])

In [18]:
Data = load_data(155000,6000,10000,5000)

24229
27078
37993
24229
27078
37993
178600


In [19]:
# paramètres 
true_y = Data[5]
training = Data[0]
y = Data[1]
batch_size=96
epochs=25
verbose1 = 1
verbose2 = 1
validation = Data[2]
val_y = Data[3]
testing = Data[4]
matrixsize11 = 6
nbfilter11 = 24
matrixsize12 = 7
nbfilter12 = 4
matrixsize21 = 6
nbfilter21 = 24
matrixsize22 = 7
nbfilter22 = 4
nbfilter1 = 64
kernelsize = 7
nbfilters2 = 64
kernel_size2 = 7
Dense1 = 128
Dense2 = 512
Dense3 = 512
Dense4 = 128
Dense5 = 64

k = matrixsize11
# init_weights
I = np.eye(k)
M = np.diag(np.ones(k-1),1) + np.diag(np.ones(k-1),-1) + np.eye(k)
I2 = np.zeros((k,k))
M2 = np.zeros((k,k))
for j in range(k):
    I2[:,j] = I[:,k-j-1]
    M2[:,j] = M[:,k-j-1]        
W = np.zeros((k,k,1,nbfilter11))
W[:,:,0,0] = I
W[:,:,0,1] = I2
W[:,:,0,2] = M
W[:,:,0,3] = M2
for j in range(4,nbfilter11):
    W[:,:,0,j] = np.random.randn(k,k)*0.2


k2 = matrixsize12
I = np.eye(k2)
M = np.diag(np.ones(k2-1),1) + np.diag(np.ones(k2-1),-1) + np.eye(k2)
I2=np.zeros((k2,k2))
M2=np.zeros((k2,k2))
for j in range(k2):
    I2[:,j] = I[:,k2-j-1]
    M2[:,j] = M[:,k2-j-1]   

Z = np.zeros((k2,k2,nbfilter11,nbfilter12))

for u in range(nbfilter12):
    Z[:,:,u,0] = I
    Z[:,:,u,1] = I2
    Z[:,:,u,2] = M
    Z[:,:,u,3] = M2
    for p in range(4,nbfilter12):
        Z[:,:,u,p]=np.random.randn(k2,k2)*0.3            

c2d1_input = keras.Input(shape=(36,36,1))
cnn2d1 = Conv2D(filters = nbfilter11, kernel_size=(k,k), padding='valid', input_shape=(36,36,1),strides=(1,1),weights=[W,np.zeros(nbfilter11)])(c2d1_input)
cnn2d1 = AveragePooling2D(pool_size=(3,3))(cnn2d1)
cnn2d1 = Conv2D(filters = nbfilter12, kernel_size = (k2, k2),strides=(1,1),padding='valid',weights=[Z,np.zeros(nbfilter12)])(cnn2d1)
cnn2d1 = Dropout(0.2)(cnn2d1)
cnn2d1 = Flatten()(cnn2d1)

c2d2_input = keras.Input(shape=(36,36,1))
cnn2d2 = Conv2D(filters = nbfilter11, kernel_size=(k,k), padding='valid', input_shape=(36,36,1),strides=(1,1),weights=[W,np.zeros(nbfilter11)])(c2d2_input)
cnn2d2 = AveragePooling2D(pool_size=(3,3))(cnn2d2)
cnn2d2 = Conv2D(filters = nbfilter12, kernel_size = (k2, k2),strides=(1,1),padding='valid',weights=[Z,np.zeros(nbfilter12)])(cnn2d2)
cnn2d2 = Dropout(0.2)(cnn2d2)
cnn2d2 = Flatten()(cnn2d2)

c1d1_input = keras.Input(shape=(36,4))
cnn1d1 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d1_input)
cnn1d1 = MaxPooling1D(pool_size=8)(cnn1d1)
cnn1d1 = Dropout(0.2)(cnn1d1)
cnn1d1 = Flatten()(cnn1d1)

c1d2_input = keras.Input(shape=(36,4))
cnn1d2 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d2_input)
cnn1d2 = MaxPooling1D(pool_size=8)(cnn1d2)
cnn1d2 = Dropout(0.2)(cnn1d2)
cnn1d2 = Flatten()(cnn1d2)

c1d3_input = keras.Input(shape=(36,4))
cnn1d3 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d3_input)
cnn1d3 = MaxPooling1D(pool_size=8)(cnn1d3)
cnn1d3 = Dropout(0.2)(cnn1d3)
cnn1d3 = Flatten()(cnn1d3)

c1d4_input = keras.Input(shape=(36,4))
cnn1d4 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d4_input)
cnn1d4 = MaxPooling1D(pool_size=8)(cnn1d4)
cnn1d4 = Dropout(0.2)(cnn1d4)
cnn1d4 = Flatten()(cnn1d4)

model21 = keras.layers.concatenate([cnn1d1,cnn1d2])
model21 = Dense(int(Dense2/2),kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model21)
model21 = BatchNormalization()(model21)
model21 = Activation('relu')(model21)
model21 = Dropout(0.2)(model21)

model22 = keras.layers.concatenate([cnn1d3,cnn1d4])
model22 = Dense(int(Dense2/2),kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model22)
model22 = BatchNormalization()(model22)
model22 = Activation('relu')(model22)
model22 = Dropout(0.2)(model22)

model2 = keras.layers.concatenate([model21,model22])
model2 = Dense(Dense2,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model2)
model2 = BatchNormalization()(model2)
model2 = Activation('relu')(model2)
model2 = Dropout(0.2)(model2)

model1 = keras.layers.concatenate([cnn2d1,cnn2d2])
model1 = Dense(Dense1,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model1)

model1 = BatchNormalization()(model1)
model1 = Activation('relu')(model1)

model = keras.layers.concatenate([model1,model2])
model = Dense(Dense3,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
model = Activation('relu')(model)
model = Dense(Dense4,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
model = Activation('relu')(model)

if Dense5>0:
    model = Dense(Dense5,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)

model = Dense(2,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
interaction_output = Activation('softmax')(model)

interaction = Model(inputs=[c2d1_input,c2d2_input,c1d1_input,c1d2_input,c1d3_input,c1d4_input],outputs=[interaction_output])

#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True) autre optimisation mais moins bons résultats
interaction.compile(loss='categorical_crossentropy', optimizer='rmsprop')

earlystopper = EarlyStopping(monitor='val_loss', patience=3, verbose = verbose1)

print('model training')
interaction.fit(training, y, batch_size, epochs, verbose = verbose2, validation_data=(validation, val_y), callbacks=[earlystopper])

# test   
print('predicting')
predictions = interaction.predict(testing)[:,1]
print(predictions)
for i,nulll in enumerate(predictions):
    predictions[i] = round(predictions[i])
print(predictions,true_y)
perfs = calculate_performance(len(predictions), predictions, true_y)
print("batch_size : ", batch_size)
print("epochs : ",epochs)
print("acc : ", perfs[0])
print("precision : ", perfs[1])
print("sensitivity : ", perfs[2])
print("specificity : ", perfs[3])
print("MCC : ", perfs[4])

model training
Train on 155000 samples, validate on 6000 samples
Epoch 1/25
155000/155000 [==============================] - 94s 606us/step - loss: 0.6430 - val_loss: 0.5885
Epoch 2/25
155000/155000 [==============================] - 89s 576us/step - loss: 0.5987 - val_loss: 0.5621
Epoch 3/25
155000/155000 [==============================] - 89s 572us/step - loss: 0.5719 - val_loss: 0.5485
Epoch 4/25
155000/155000 [==============================] - 89s 576us/step - loss: 0.5528 - val_loss: 0.5354
Epoch 5/25
155000/155000 [==============================] - 90s 579us/step - loss: 0.5362 - val_loss: 0.5277
Epoch 6/25
155000/155000 [==============================] - 89s 575us/step - loss: 0.5219 - val_loss: 0.5235
Epoch 7/25
155000/155000 [==============================] - 89s 576us/step - loss: 0.5104 - val_loss: 0.5162
Epoch 8/25
155000/155000 [==============================] - 91s 587us/step - loss: 0.4993 - val_loss: 0.5098
Epoch 9/25
155000/155000 [==============================] - 90s

In [20]:
interaction.save_weights("model_rna-rna_deep_livrable2.h5")

In [21]:
interaction.get_weights()

[array([[[-3.34762126e-01,  6.62024226e-03,  6.80660978e-02, ...,
           1.13259718e-01,  8.45903084e-02, -8.24314654e-02],
         [ 2.09554955e-01, -5.31359613e-01,  1.72473907e-01, ...,
          -4.34647173e-01,  8.36880729e-02, -1.97610766e-01],
         [-4.17283952e-01,  2.04907626e-01, -9.14868191e-02, ...,
          -2.05904350e-01, -3.17810476e-02,  3.36416572e-01],
         [ 1.84245016e-02,  2.47794062e-01, -2.16503963e-01, ...,
           1.95892483e-01,  2.38910634e-02,  1.86597526e-01]],
 
        [[-2.79279053e-01,  2.83853412e-01, -3.15817118e-01, ...,
          -2.88706552e-02,  1.63789690e-01, -2.44182512e-01],
         [-1.63828041e-02, -1.43336713e-01, -1.07118068e-02, ...,
          -3.05834785e-02, -6.50352538e-02, -1.47439942e-01],
         [-2.34915033e-01, -4.89945620e-01,  7.36669777e-03, ...,
           1.48661911e-01,  2.67443329e-01,  6.54133111e-02],
         [-2.58765817e-01, -5.14487363e-02,  1.93044752e-01, ...,
           9.72140431e-02,  2.47734

### Test benchmark

In [22]:
Data[6]

array([['cugggggcccuguguuccauccaauagaugacuaugagcauccacuucuguauuugccagguacuggcauagccucacaagagagagcuauaucaggguccugucucucaauugagauucccucuucucaaaugacucuaguuugugucaaauggacauaaaauuaaccaacacacauguccacuccucc',
        'gcgagggcaaagggaaagaggcuagcaguggucauugucccgacggugugguggucuguuggccgaggugagucuugggggcucguccggcccugucguccuucgggaaggcgcguguuggggccugccggagugccgagguggguacccuggcggugggauuaaccccgcgcgcgugucccgguguggcggugggggcuccgguc',
        '[9.88065706e-01 1.03804032e-03 1.95950809e-03 3.11392511e-03\n 1.00680832e-02 6.60524101e-02 9.88956688e-01 3.55357253e-01\n 2.85669339e-01 9.55655807e-02 8.90511539e-02 6.07468231e-01\n 6.74640695e-01 9.88717154e-01 2.70526513e-01 2.68156497e-01\n 2.67674937e-01 8.05363926e-02 7.47679663e-01 4.69604439e-01\n 7.24913067e-01 7.06569693e-01 9.13589172e-01 9.18858526e-01\n 9.27760891e-01 9.93441327e-01 4.38210107e-01 2.77524221e-01\n 4.22604353e-04 4.01100283e-04 9.71073691e-04 9.85640894e-03\n 8.07571163e-01 9.98283893e-01 7.99518177e-01 7.89480572e-01]',
        ...

In [23]:
benchmark = Data[6]
bdd = benchmark
ind_positifs = np.argwhere(bdd[:,6]==1)
bdd = bdd[ind_positifs]
l = len(bdd)
writer = csv.writer(open("genomes/benchmark_interactions.csv", 'w',newline=''),delimiter="\t")
for i in range(l):
    writer.writerow(bdd[i,:])

In [ ]:
r = 0

seq1 = bdd[r,0]
seq2 = bdd[r,1]
dim = len(seq1)
dim2 = len(seq2)
matrice = np.zeros((1,dim,dim2,1)) 
matrice1 = np.zeros((1,dim,dim2,1))
matrice2 = np.zeros((1,dim,4))
matrice3 = np.zeros((1,dim,4))


proba1 = RNA.pfl_fold_up(seq1,1,len(seq1),len(seq1))[1:,1]
proba2 = RNA.pfl_fold_up(seq2,1,len(seq2),len(seq2))[1:,1]
for j in range(dim):
    for k in range(dim2):
        if (seq1[j]=='a' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='a'):
            matrice[i,j,k,0] = 1
        elif (seq1[j]=='g' and seq2[k]=='c') or (seq1[j]=='c' and seq2[k]=='g'):
            matrice[i,j,k,0] = 1
        elif (seq1[j]=='g' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='g'):
            matrice[i,j,k,0] = 1
        matrice1[i,j,k,0] = proba1[j]+proba2[k]
for j in range(dim):
    if seq1[j]=='a':
        matrice2[i,j,0] = 1
    elif seq1[j]=='u':
        matrice2[i,j,1] = 1
    elif seq1[j]=='g':
        matrice2[i,j,2] = 1
    elif seq1[j]=='c':
        matrice2[i,j,3] = 1
for j in range(dim2):
    if seq2[j]=='a':
        matrice3[i,j,0] = 1
    elif seq2[j]=='u':
        matrice3[i,j,1] = 1
    elif seq2[j]=='g':
        matrice3[i,j,2] = 1
    elif seq2[j]=='c':
        matrice3[i,j,3] = 1

resultats = np.zeros((dim,dim2))
        
for i in range(dim):
    for j in range(dim2):        
        test = []
        test.append(matrice[:,i:i+36,j:j+36,:])
        test.append(matrice1[:,i:i+36,j:j+36,:])
        test.append(matrice2[:,i:i+36])
        rev = np.zeros((1,36,4))
        for q in range(36):
            rev[:,36-q-1,:] = matrice2[:,q,:]
        test.append(rev)
        test.append(matrice3[:,j:j+36])
        rev2 = np.zeros((1,36,4))
        for q in range(36):
            rev2[:,36-q-1,:] = matrice3[:,q,:]
        test.append(rev2)

        resultats[i,j] = interaction.predict(test)[:,1]